In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from PIL import Image
import os
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
import zipfile

import torch.nn.functional as F
from rich.console import Console
from rich.progress import Progress, BarColumn, TimeElapsedColumn, TimeRemainingColumn
from rich.table import Table

import pandas as pd
from torch.cuda.amp import autocast, GradScaler


from sklearn.metrics import precision_score, recall_score, f1_score, multilabel_confusion_matrix
import seaborn as sns
import numpy as np

In [ ]:
# Install Git in case it's not available in your environment
!apt-get install git

# Clone the repository from GitHub
!git clone https://github.com/Samin1362/CSE499-MerakiNexus-AI_And_Defi.git

In [ ]:
import os
import matplotlib.pyplot as plt
from torchvision import datasets

# Define dataset path
dataset_path = "/content/CSE499-MerakiNexus-AI_And_Defi/sentiment_model/sentiment_dataset"  # Modify with your dataset path

# Load the datasets
train_dataset = datasets.ImageFolder(root=f"{dataset_path}/train")
val_dataset = datasets.ImageFolder(root=f"{dataset_path}/val")
test_dataset = datasets.ImageFolder(root=f"{dataset_path}/test")

# Get class names and their corresponding folder paths
class_names = train_dataset.classes
train_image_counts = []
val_image_counts = []
test_image_counts = []

# Count the number of images in each class folder for all datasets
for class_name in class_names:
    # Count for training set
    train_class_folder = os.path.join(train_dataset.root, class_name)
    train_image_count = len(os.listdir(train_class_folder))  # Count files in each class folder
    train_image_counts.append(train_image_count)

    # Count for validation set
    val_class_folder = os.path.join(val_dataset.root, class_name)
    val_image_count = len(os.listdir(val_class_folder))  # Count files in each class folder
    val_image_counts.append(val_image_count)

    # Count for testing set
    test_class_folder = os.path.join(test_dataset.root, class_name)
    test_image_count = len(os.listdir(test_class_folder))  # Count files in each class folder
    test_image_counts.append(test_image_count)

# Plotting the class distribution for training, validation, and testing sets side by side
fig, axes = plt.subplots(1, 4, figsize=(24, 6))

# Plot for training dataset
axes[0].bar(class_names, train_image_counts, color='skyblue')
axes[0].set_title('Training Image Count per Class', fontsize=16)
axes[0].set_xlabel('Class', fontsize=14)
axes[0].set_ylabel('Number of Images', fontsize=14)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

# Plot for validation dataset
axes[1].bar(class_names, val_image_counts, color='lightgreen')
axes[1].set_title('Validation Image Count per Class', fontsize=16)
axes[1].set_xlabel('Class', fontsize=14)
axes[1].set_ylabel('Number of Images', fontsize=14)
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

# Plot for testing dataset
axes[2].bar(class_names, test_image_counts, color='lightcoral')
axes[2].set_title('Testing Image Count per Class', fontsize=16)
axes[2].set_xlabel('Class', fontsize=14)
axes[2].set_ylabel('Number of Images', fontsize=14)
axes[2].tick_params(axis='x', rotation=45)
axes[2].grid(axis='y', linestyle='--', alpha=0.7)

# Plot for combined view (Training, Validation, and Testing)
axes[3].bar(class_names, train_image_counts, color='skyblue', label='Train')
axes[3].bar(class_names, val_image_counts, color='lightgreen', label='Validation', alpha=0.7)
axes[3].bar(class_names, test_image_counts, color='lightcoral', label='Test', alpha=0.7)
axes[3].set_title('Combined Image Count per Class (Train, Validation, Test)', fontsize=16)
axes[3].set_xlabel('Class', fontsize=14)
axes[3].set_ylabel('Number of Images', fontsize=14)
axes[3].tick_params(axis='x', rotation=45)
axes[3].legend()
axes[3].grid(axis='y', linestyle='--', alpha=0.7)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

# Mapping of art styles (classes) to sentiment categories (binary labels)
class_to_main_class = {
    "fauvism": [1, 0, 0, 0],               # Emotion Through Color
    "expressionism": [1, 0, 0, 0],
    "abstract_expressionism": [1, 0, 0, 0],

    "cubism": [0, 1, 0, 0],                # Visual Complexity
    "surrealism": [0, 1, 0, 0],
    "op_art": [0, 1, 0, 0],

    "futurism": [0, 0, 1, 0],              # Movement and Flow
    "baroque": [0, 0, 1, 0],
    "art_nouveau": [0, 0, 1, 0],

    "realism": [0, 0, 0, 1],               # Facial Expressions and Human Emotion
    "romanticism": [0, 0, 0, 1]
}

# Custom dataset class for sentiment model
class CustomSentimentDataset(datasets.ImageFolder):
    def __init__(self, root_dir, transform=None):
        super().__init__(root_dir, transform=transform)
        self.class_to_main_class = class_to_main_class

    def __getitem__(self, idx):
        img_path, class_idx = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(self.class_to_main_class[self.classes[class_idx]], dtype=torch.float)

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Path to dataset (modify to your local directory)
dataset_path = "/content/CSE499-MerakiNexus-AI_And_Defi/sentiment_model/sentiment_dataset"

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load datasets
train_data = CustomSentimentDataset(root_dir=f"{dataset_path}/train", transform=transform)
val_data = CustomSentimentDataset(root_dir=f"{dataset_path}/val", transform=transform)
test_data = CustomSentimentDataset(root_dir=f"{dataset_path}/test", transform=transform)

# Data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define sentiment categories
sentiment_categories = ["Emotion Through Color", "Visual Complexity", "Movement and Flow", "Facial Expressions and Human Emotion"]

# Define model based on ResNet50 with custom heads for the 4 sentiment classes
class SentimentModelWithResNet50(nn.Module):
    def __init__(self, num_classes=4):
        super(SentimentModelWithResNet50, self).__init__()
        self.resnet = models.resnet50(pretrained=True)

        # Get the in_features from ResNet50's final fully connected layer
        in_features = self.resnet.fc.in_features

        # Remove the final fully connected layer (so that we can add custom heads)
        self.resnet.fc = nn.Identity()

        # Custom heads for the 4 sentiment classes
        self.emotion_color_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)  # Output for Emotion Through Color
        )

        self.visual_complexity_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)  # Output for Visual Complexity
        )

        self.movement_flow_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)  # Output for Movement and Flow
        )

        self.human_emotion_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)  # Output for Facial Expressions and Human Emotion
        )

    def forward(self, x):
        features = self.resnet(x)
        emotion_color = self.emotion_color_head(features)
        visual_complexity = self.visual_complexity_head(features)
        movement_flow = self.movement_flow_head(features)
        human_emotion = self.human_emotion_head(features)
        return emotion_color, visual_complexity, movement_flow, human_emotion

# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resNet50_model = SentimentModelWithResNet50(num_classes=4).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(resNet50_model.parameters(), lr=1e-4)

# Training setup
num_epochs = 25
start_time = time.time()
train_losses = []
val_accuracies = []

# Training loop
for epoch in range(num_epochs):
    resNet50_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    epoch_start_time = time.time()

    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training", unit="batch") as tepoch:
        for images, labels in tepoch:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            emotion_color, visual_complexity, movement_flow, human_emotion = resNet50_model(images)

            loss1 = criterion(emotion_color.squeeze(), labels[:, 0])
            loss2 = criterion(visual_complexity.squeeze(), labels[:, 1])
            loss3 = criterion(movement_flow.squeeze(), labels[:, 2])
            loss4 = criterion(human_emotion.squeeze(), labels[:, 3])

            total_loss = loss1 + loss2 + loss3 + loss4
            total_loss.backward()
            optimizer.step()

            running_loss += total_loss.item()

            predictions = torch.cat([
                torch.sigmoid(emotion_color),
                torch.sigmoid(visual_complexity),
                torch.sigmoid(movement_flow),
                torch.sigmoid(human_emotion)
            ], dim=1)

            predicted_labels = (predictions > 0.5).float()
            correct_train += (predicted_labels == labels).sum().item()
            total_train += labels.size(0) * labels.size(1)

            tepoch.set_postfix(loss=total_loss.item())

    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    print(f"Epoch {epoch+1}/{num_epochs} completed in {epoch_duration:.2f} seconds.")
    print(f"Epoch Loss: {running_loss / len(train_loader)}")

    train_accuracy = correct_train / total_train if total_train > 0 else 0
    print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
    train_losses.append(running_loss / len(train_loader))

    # Validation phase
    resNet50_model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} Validation", leave=False):
            images, labels = images.to(device), labels.to(device)
            emotion_color, visual_complexity, movement_flow, human_emotion = resNet50_model(images)

            pred1 = torch.sigmoid(emotion_color)
            pred2 = torch.sigmoid(visual_complexity)
            pred3 = torch.sigmoid(movement_flow)
            pred4 = torch.sigmoid(human_emotion)

            correct_val += (pred1 > 0.5).float().eq(labels[:, 0].unsqueeze(1)).sum().item()
            correct_val += (pred2 > 0.5).float().eq(labels[:, 1].unsqueeze(1)).sum().item()
            correct_val += (pred3 > 0.5).float().eq(labels[:, 2].unsqueeze(1)).sum().item()
            correct_val += (pred4 > 0.5).float().eq(labels[:, 3].unsqueeze(1)).sum().item()
            total_val += labels.size(0) * 4

    val_accuracy = correct_val / total_val
    print(f"Epoch {epoch+1}/{num_epochs} Validation Accuracy: {val_accuracy * 100:.2f}%")
    val_accuracies.append(val_accuracy)

# Total training time
total_training_time = time.time() - start_time
print(f"\nTotal Training Time: {total_training_time:.2f} seconds ({(total_training_time / 60):.2f} minutes)")

# Plotting Loss and Accuracy
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', color='blue')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='green')
plt.title('Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Define sentiment categories
sentiment_categories = ["Emotion Through Color", "Visual Complexity", "Movement and Flow", "Facial Expressions and Human Emotion"]

# Set the model to evaluation mode
resNet50_model.eval()

# Initialize variables to hold predictions and ground truths
all_predictions = []
all_labels = []

# Iterate over the test dataset
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass through the model
        emotion_color, visual_complexity, movement_flow, human_emotion = resNet50_model(images)

        # Convert logits to probabilities using sigmoid
        pred_emotion_color = torch.sigmoid(emotion_color)
        pred_visual_complexity = torch.sigmoid(visual_complexity)
        pred_movement_flow = torch.sigmoid(movement_flow)
        pred_human_emotion = torch.sigmoid(human_emotion)

        # Binarize predictions (thresholding at 0.5)
        bin_emotion_color = (pred_emotion_color > 0.5).float()
        bin_visual_complexity = (pred_visual_complexity > 0.5).float()
        bin_movement_flow = (pred_movement_flow > 0.5).float()
        bin_human_emotion = (pred_human_emotion > 0.5).float()

        # Concatenate predictions for multi-label output
        batch_preds = torch.cat([bin_emotion_color, bin_visual_complexity, bin_movement_flow, bin_human_emotion], dim=1)
        all_predictions.append(batch_preds)
        all_labels.append(labels)

# Convert to NumPy arrays
all_predictions = torch.cat(all_predictions, dim=0).cpu().numpy()
all_labels = torch.cat(all_labels, dim=0).cpu().numpy()

# Calculate accuracy
accuracy = (all_predictions == all_labels).sum() / all_labels.size

# Compute evaluation metrics
precision = precision_score(all_labels, all_predictions, average='samples')
recall = recall_score(all_labels, all_predictions, average='samples')
f1 = f1_score(all_labels, all_predictions, average='samples')

# Multi-label confusion matrices
mcm = multilabel_confusion_matrix(all_labels, all_predictions)

# Plot confusion matrices for each class
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
for i, ax in enumerate(axes.flatten()):
    sns.heatmap(mcm[i], annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred Negative', 'Pred Positive'],
                yticklabels=['True Negative', 'True Positive'],
                ax=ax)
    ax.set_title(f'Confusion Matrix for {sentiment_categories[i]}')

plt.tight_layout()
plt.show()

# Display evaluation metrics
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define sentiment categories
sentiment_categories = ["Emotion Through Color", "Visual Complexity", "Movement and Flow", "Facial Expressions and Human Emotion"]

# Define model based on ResNet101 with custom heads for the 4 sentiment classes
class SentimentModelWithResNet101(nn.Module):
    def __init__(self, num_classes=4):
        super(SentimentModelWithResNet101, self).__init__()
        self.resnet = models.resnet101(pretrained=True)

        # Get the in_features from ResNet101's final fully connected layer
        in_features = self.resnet.fc.in_features

        # Remove the final fully connected layer (so that we can add custom heads)
        self.resnet.fc = nn.Identity()

        # Custom heads for the 4 sentiment classes
        self.emotion_color_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)  # Output for Emotion Through Color
        )

        self.visual_complexity_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)  # Output for Visual Complexity
        )

        self.movement_flow_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)  # Output for Movement and Flow
        )

        self.human_emotion_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)  # Output for Facial Expressions and Human Emotion
        )

    def forward(self, x):
        features = self.resnet(x)
        emotion_color = self.emotion_color_head(features)
        visual_complexity = self.visual_complexity_head(features)
        movement_flow = self.movement_flow_head(features)
        human_emotion = self.human_emotion_head(features)
        return emotion_color, visual_complexity, movement_flow, human_emotion

# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resNet101_model = SentimentModelWithResNet101(num_classes=4).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(resNet101_model.parameters(), lr=1e-4)

# Training setup
num_epochs = 25
start_time = time.time()
train_losses = []
val_accuracies = []

# Training loop
for epoch in range(num_epochs):
    resNet101_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    epoch_start_time = time.time()

    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training", unit="batch") as tepoch:
        for images, labels in tepoch:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            emotion_color, visual_complexity, movement_flow, human_emotion = resNet101_model(images)

            loss1 = criterion(emotion_color.squeeze(), labels[:, 0])
            loss2 = criterion(visual_complexity.squeeze(), labels[:, 1])
            loss3 = criterion(movement_flow.squeeze(), labels[:, 2])
            loss4 = criterion(human_emotion.squeeze(), labels[:, 3])

            total_loss = loss1 + loss2 + loss3 + loss4
            total_loss.backward()
            optimizer.step()

            running_loss += total_loss.item()

            predictions = torch.cat([
                torch.sigmoid(emotion_color),
                torch.sigmoid(visual_complexity),
                torch.sigmoid(movement_flow),
                torch.sigmoid(human_emotion)
            ], dim=1)

            predicted_labels = (predictions > 0.5).float()
            correct_train += (predicted_labels == labels).sum().item()
            total_train += labels.size(0) * labels.size(1)

            tepoch.set_postfix(loss=total_loss.item())

    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    print(f"Epoch {epoch+1}/{num_epochs} completed in {epoch_duration:.2f} seconds.")
    print(f"Epoch Loss: {running_loss / len(train_loader)}")

    train_accuracy = correct_train / total_train if total_train > 0 else 0
    print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
    train_losses.append(running_loss / len(train_loader))

    # Validation phase
    resNet101_model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} Validation", leave=False):
            images, labels = images.to(device), labels.to(device)
            emotion_color, visual_complexity, movement_flow, human_emotion = resNet101_model(images)

            pred1 = torch.sigmoid(emotion_color)
            pred2 = torch.sigmoid(visual_complexity)
            pred3 = torch.sigmoid(movement_flow)
            pred4 = torch.sigmoid(human_emotion)

            correct_val += (pred1 > 0.5).float().eq(labels[:, 0].unsqueeze(1)).sum().item()
            correct_val += (pred2 > 0.5).float().eq(labels[:, 1].unsqueeze(1)).sum().item()
            correct_val += (pred3 > 0.5).float().eq(labels[:, 2].unsqueeze(1)).sum().item()
            correct_val += (pred4 > 0.5).float().eq(labels[:, 3].unsqueeze(1)).sum().item()
            total_val += labels.size(0) * 4

    val_accuracy = correct_val / total_val
    print(f"Epoch {epoch+1}/{num_epochs} Validation Accuracy: {val_accuracy * 100:.2f}%")
    val_accuracies.append(val_accuracy)

# Total training time
total_training_time = time.time() - start_time
print(f"\nTotal Training Time: {total_training_time:.2f} seconds ({(total_training_time / 60):.2f} minutes)")

# Plotting Loss and Accuracy
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', color='blue')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='green')
plt.title('Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Define sentiment categories
sentiment_categories = ["Emotion Through Color", "Visual Complexity", "Movement and Flow", "Facial Expressions and Human Emotion"]

# Set the model to evaluation mode
resNet101_model.eval()

# Initialize variables to hold predictions and ground truths
all_predictions = []
all_labels = []

# Iterate over the test dataset
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass through the model
        emotion_color, visual_complexity, movement_flow, human_emotion = resNet101_model(images)

        # Convert logits to probabilities using sigmoid
        pred_emotion_color = torch.sigmoid(emotion_color)
        pred_visual_complexity = torch.sigmoid(visual_complexity)
        pred_movement_flow = torch.sigmoid(movement_flow)
        pred_human_emotion = torch.sigmoid(human_emotion)

        # Binarize predictions (thresholding at 0.5)
        bin_emotion_color = (pred_emotion_color > 0.5).float()
        bin_visual_complexity = (pred_visual_complexity > 0.5).float()
        bin_movement_flow = (pred_movement_flow > 0.5).float()
        bin_human_emotion = (pred_human_emotion > 0.5).float()

        # Concatenate predictions for multi-label output
        batch_preds = torch.cat([bin_emotion_color, bin_visual_complexity, bin_movement_flow, bin_human_emotion], dim=1)
        all_predictions.append(batch_preds)
        all_labels.append(labels)

# Convert to NumPy arrays
all_predictions = torch.cat(all_predictions, dim=0).cpu().numpy()
all_labels = torch.cat(all_labels, dim=0).cpu().numpy()

# Calculate accuracy
accuracy = (all_predictions == all_labels).sum() / all_labels.size

# Compute evaluation metrics
precision = precision_score(all_labels, all_predictions, average='samples')
recall = recall_score(all_labels, all_predictions, average='samples')
f1 = f1_score(all_labels, all_predictions, average='samples')

# Multi-label confusion matrices
mcm = multilabel_confusion_matrix(all_labels, all_predictions)

# Plot confusion matrices for each class
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
for i, ax in enumerate(axes.flatten()):
    sns.heatmap(mcm[i], annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred Negative', 'Pred Positive'],
                yticklabels=['True Negative', 'True Positive'],
                ax=ax)
    ax.set_title(f'Confusion Matrix for {sentiment_categories[i]}')

plt.tight_layout()
plt.show()

# Display evaluation metrics
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")


In [ ]:
!pip install timm

import torch
import torch.nn as nn
import torch.optim as optim
import timm
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define sentiment categories
sentiment_categories = ["Emotion Through Color", "Visual Complexity", "Movement and Flow", "Facial Expressions and Human Emotion"]

# Define ViT-based sentiment model with 4 custom heads
class SentimentModelWithViT(nn.Module):
    def __init__(self, backbone_name="vit_base_patch16_224", num_classes=4):
        super(SentimentModelWithViT, self).__init__()
        self.vit = timm.create_model(backbone_name, pretrained=True)

        # Extract the feature dimension from ViT head
        in_features = self.vit.head.in_features
        self.vit.head = nn.Identity()  # Remove the classification head

        # Four sentiment heads
        self.emotion_color_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.visual_complexity_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.movement_flow_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.human_emotion_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        features = self.vit(x)
        return (
            self.emotion_color_head(features),
            self.visual_complexity_head(features),
            self.movement_flow_head(features),
            self.human_emotion_head(features)
        )

# Initialize model, loss, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vit_model = SentimentModelWithViT().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(vit_model.parameters(), lr=1e-4)

# Training settings
num_epochs = 25
start_time = time.time()
train_losses = []
val_accuracies = []

# Training loop
for epoch in range(num_epochs):
    vit_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    epoch_start_time = time.time()

    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training", unit="batch") as tepoch:
        for images, labels in tepoch:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            emotion_color, visual_complexity, movement_flow, human_emotion = vit_model(images)

            # Loss calculation
            loss1 = criterion(emotion_color.squeeze(), labels[:, 0])
            loss2 = criterion(visual_complexity.squeeze(), labels[:, 1])
            loss3 = criterion(movement_flow.squeeze(), labels[:, 2])
            loss4 = criterion(human_emotion.squeeze(), labels[:, 3])
            total_loss = loss1 + loss2 + loss3 + loss4
            total_loss.backward()
            optimizer.step()
            running_loss += total_loss.item()

            # Accuracy calculation
            predictions = torch.cat([
                torch.sigmoid(emotion_color),
                torch.sigmoid(visual_complexity),
                torch.sigmoid(movement_flow),
                torch.sigmoid(human_emotion)
            ], dim=1)

            predicted_labels = (predictions > 0.5).float()
            correct_train += (predicted_labels == labels).sum().item()
            total_train += labels.size(0) * labels.size(1)

            tepoch.set_postfix(loss=total_loss.item())

    epoch_duration = time.time() - epoch_start_time
    print(f"Epoch {epoch+1} completed in {epoch_duration:.2f} seconds.")
    print(f"Epoch Loss: {running_loss / len(train_loader)}")
    train_accuracy = correct_train / total_train if total_train > 0 else 0
    print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
    train_losses.append(running_loss / len(train_loader))

    # Validation
    vit_model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} Validation", leave=False):
            images, labels = images.to(device), labels.to(device)
            emotion_color, visual_complexity, movement_flow, human_emotion = vit_model(images)

            pred1 = torch.sigmoid(emotion_color)
            pred2 = torch.sigmoid(visual_complexity)
            pred3 = torch.sigmoid(movement_flow)
            pred4 = torch.sigmoid(human_emotion)

            correct_val += (pred1 > 0.5).float().eq(labels[:, 0].unsqueeze(1)).sum().item()
            correct_val += (pred2 > 0.5).float().eq(labels[:, 1].unsqueeze(1)).sum().item()
            correct_val += (pred3 > 0.5).float().eq(labels[:, 2].unsqueeze(1)).sum().item()
            correct_val += (pred4 > 0.5).float().eq(labels[:, 3].unsqueeze(1)).sum().item()
            total_val += labels.size(0) * 4

    val_accuracy = correct_val / total_val
    print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
    val_accuracies.append(val_accuracy)

# Training complete
total_training_time = time.time() - start_time
print(f"\nTotal Training Time: {total_training_time:.2f} seconds ({total_training_time / 60:.2f} minutes)")

# Plotting
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', color='blue')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='green')
plt.title('Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Define sentiment categories
sentiment_categories = ["Emotion Through Color", "Visual Complexity", "Movement and Flow", "Facial Expressions and Human Emotion"]

# Set the model to evaluation mode
vit_model.eval()

# Initialize variables to hold predictions and ground truths
all_predictions = []
all_labels = []

# Iterate over the test dataset
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass through the model
        emotion_color, visual_complexity, movement_flow, human_emotion = vit_model(images)

        # Convert logits to probabilities using sigmoid
        pred_emotion_color = torch.sigmoid(emotion_color)
        pred_visual_complexity = torch.sigmoid(visual_complexity)
        pred_movement_flow = torch.sigmoid(movement_flow)
        pred_human_emotion = torch.sigmoid(human_emotion)

        # Binarize predictions (thresholding at 0.5)
        bin_emotion_color = (pred_emotion_color > 0.5).float()
        bin_visual_complexity = (pred_visual_complexity > 0.5).float()
        bin_movement_flow = (pred_movement_flow > 0.5).float()
        bin_human_emotion = (pred_human_emotion > 0.5).float()

        # Concatenate predictions for multi-label output
        batch_preds = torch.cat([bin_emotion_color, bin_visual_complexity, bin_movement_flow, bin_human_emotion], dim=1)
        all_predictions.append(batch_preds)
        all_labels.append(labels)

# Convert to NumPy arrays
all_predictions = torch.cat(all_predictions, dim=0).cpu().numpy()
all_labels = torch.cat(all_labels, dim=0).cpu().numpy()

# Calculate accuracy
accuracy = (all_predictions == all_labels).sum() / all_labels.size

# Compute evaluation metrics
precision = precision_score(all_labels, all_predictions, average='samples')
recall = recall_score(all_labels, all_predictions, average='samples')
f1 = f1_score(all_labels, all_predictions, average='samples')

# Multi-label confusion matrices
mcm = multilabel_confusion_matrix(all_labels, all_predictions)

# Plot confusion matrices for each class
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
for i, ax in enumerate(axes.flatten()):
    sns.heatmap(mcm[i], annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred Negative', 'Pred Positive'],
                yticklabels=['True Negative', 'True Positive'],
                ax=ax)
    ax.set_title(f'Confusion Matrix for {sentiment_categories[i]}')

plt.tight_layout()
plt.show()

# Display evaluation metrics
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import timm
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define sentiment categories
sentiment_categories = ["Emotion Through Color", "Visual Complexity", "Movement and Flow", "Facial Expressions and Human Emotion"]

# Define model based on EfficientNet with custom heads for the 4 sentiment classes
class SentimentModelWithEfficientNet(nn.Module):
    def __init__(self, backbone_name="efficientnet_b0", num_classes=4):
        super(SentimentModelWithEfficientNet, self).__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=True)
        in_features = self.backbone.classifier.in_features  # Get input size of the head
        self.backbone.classifier = nn.Identity()  # Remove the classification layer

        # Four custom heads for each sentiment dimension
        self.emotion_color_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.visual_complexity_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.movement_flow_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.human_emotion_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        features = self.backbone(x)
        return (
            self.emotion_color_head(features),
            self.visual_complexity_head(features),
            self.movement_flow_head(features),
            self.human_emotion_head(features)
        )

# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
effnet_model = SentimentModelWithEfficientNet().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(effnet_model.parameters(), lr=1e-4)

# Training setup
num_epochs = 25
start_time = time.time()
train_losses = []
val_accuracies = []

# Training loop
for epoch in range(num_epochs):
    effnet_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    epoch_start_time = time.time()

    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training", unit="batch") as tepoch:
        for images, labels in tepoch:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            ec, vc, mf, he = effnet_model(images)

            loss1 = criterion(ec.squeeze(), labels[:, 0])
            loss2 = criterion(vc.squeeze(), labels[:, 1])
            loss3 = criterion(mf.squeeze(), labels[:, 2])
            loss4 = criterion(he.squeeze(), labels[:, 3])
            total_loss = loss1 + loss2 + loss3 + loss4

            total_loss.backward()
            optimizer.step()
            running_loss += total_loss.item()

            predictions = torch.cat([
                torch.sigmoid(ec),
                torch.sigmoid(vc),
                torch.sigmoid(mf),
                torch.sigmoid(he)
            ], dim=1)

            predicted_labels = (predictions > 0.5).float()
            correct_train += (predicted_labels == labels).sum().item()
            total_train += labels.size(0) * labels.size(1)

            tepoch.set_postfix(loss=total_loss.item())

    epoch_duration = time.time() - epoch_start_time
    print(f"Epoch {epoch+1} completed in {epoch_duration:.2f} seconds.")
    print(f"Epoch Loss: {running_loss / len(train_loader)}")
    train_accuracy = correct_train / total_train
    print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
    train_losses.append(running_loss / len(train_loader))

    # Validation
    effnet_model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} Validation", leave=False):
            images, labels = images.to(device), labels.to(device)
            ec, vc, mf, he = effnet_model(images)

            pred1 = torch.sigmoid(ec)
            pred2 = torch.sigmoid(vc)
            pred3 = torch.sigmoid(mf)
            pred4 = torch.sigmoid(he)

            correct_val += (pred1 > 0.5).float().eq(labels[:, 0].unsqueeze(1)).sum().item()
            correct_val += (pred2 > 0.5).float().eq(labels[:, 1].unsqueeze(1)).sum().item()
            correct_val += (pred3 > 0.5).float().eq(labels[:, 2].unsqueeze(1)).sum().item()
            correct_val += (pred4 > 0.5).float().eq(labels[:, 3].unsqueeze(1)).sum().item()
            total_val += labels.size(0) * 4

    val_accuracy = correct_val / total_val
    print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
    val_accuracies.append(val_accuracy)

# Training complete
total_training_time = time.time() - start_time
print(f"\nTotal Training Time: {total_training_time:.2f} seconds ({total_training_time / 60:.2f} minutes)")

# Plotting
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', color='blue')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='green')
plt.title('Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Define sentiment categories
sentiment_categories = ["Emotion Through Color", "Visual Complexity", "Movement and Flow", "Facial Expressions and Human Emotion"]

# Set the model to evaluation mode
effnet_model.eval()

# Initialize variables to hold predictions and ground truths
all_predictions = []
all_labels = []

# Iterate over the test dataset
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass through the model
        emotion_color, visual_complexity, movement_flow, human_emotion = effnet_model(images)

        # Convert logits to probabilities using sigmoid
        pred_emotion_color = torch.sigmoid(emotion_color)
        pred_visual_complexity = torch.sigmoid(visual_complexity)
        pred_movement_flow = torch.sigmoid(movement_flow)
        pred_human_emotion = torch.sigmoid(human_emotion)

        # Binarize predictions (thresholding at 0.5)
        bin_emotion_color = (pred_emotion_color > 0.5).float()
        bin_visual_complexity = (pred_visual_complexity > 0.5).float()
        bin_movement_flow = (pred_movement_flow > 0.5).float()
        bin_human_emotion = (pred_human_emotion > 0.5).float()

        # Concatenate predictions for multi-label output
        batch_preds = torch.cat([bin_emotion_color, bin_visual_complexity, bin_movement_flow, bin_human_emotion], dim=1)
        all_predictions.append(batch_preds)
        all_labels.append(labels)

# Convert to NumPy arrays
all_predictions = torch.cat(all_predictions, dim=0).cpu().numpy()
all_labels = torch.cat(all_labels, dim=0).cpu().numpy()

# Calculate accuracy
accuracy = (all_predictions == all_labels).sum() / all_labels.size

# Compute evaluation metrics
precision = precision_score(all_labels, all_predictions, average='samples')
recall = recall_score(all_labels, all_predictions, average='samples')
f1 = f1_score(all_labels, all_predictions, average='samples')

# Multi-label confusion matrices
mcm = multilabel_confusion_matrix(all_labels, all_predictions)

# Plot confusion matrices for each class
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
for i, ax in enumerate(axes.flatten()):
    sns.heatmap(mcm[i], annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred Negative', 'Pred Positive'],
                yticklabels=['True Negative', 'True Positive'],
                ax=ax)
    ax.set_title(f'Confusion Matrix for {sentiment_categories[i]}')

plt.tight_layout()
plt.show()

# Display evaluation metrics
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define sentiment categories
sentiment_categories = ["Emotion Through Color", "Visual Complexity", "Movement and Flow", "Facial Expressions and Human Emotion"]

# Define AlexNet-based model with 4 custom heads
class SentimentModelWithAlexNet(nn.Module):
    def __init__(self, num_classes=4):
        super(SentimentModelWithAlexNet, self).__init__()
        self.backbone = models.alexnet(pretrained=True)
        in_features = self.backbone.classifier[6].in_features

        # Remove the original classification head
        self.backbone.classifier[6] = nn.Identity()

        # Define 4 separate heads for each sentiment category
        self.emotion_color_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.visual_complexity_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.movement_flow_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.human_emotion_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        features = self.backbone(x)
        return (
            self.emotion_color_head(features),
            self.visual_complexity_head(features),
            self.movement_flow_head(features),
            self.human_emotion_head(features)
        )

# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
alexnet_model = SentimentModelWithAlexNet().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(alexnet_model.parameters(), lr=1e-4)

# Training setup
num_epochs = 25
start_time = time.time()
train_losses = []
val_accuracies = []

# Training loop
for epoch in range(num_epochs):
    alexnet_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    epoch_start_time = time.time()

    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training", unit="batch") as tepoch:
        for images, labels in tepoch:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            ec, vc, mf, he = alexnet_model(images)

            loss1 = criterion(ec.squeeze(), labels[:, 0])
            loss2 = criterion(vc.squeeze(), labels[:, 1])
            loss3 = criterion(mf.squeeze(), labels[:, 2])
            loss4 = criterion(he.squeeze(), labels[:, 3])
            total_loss = loss1 + loss2 + loss3 + loss4

            total_loss.backward()
            optimizer.step()
            running_loss += total_loss.item()

            predictions = torch.cat([
                torch.sigmoid(ec),
                torch.sigmoid(vc),
                torch.sigmoid(mf),
                torch.sigmoid(he)
            ], dim=1)

            predicted_labels = (predictions > 0.5).float()
            correct_train += (predicted_labels == labels).sum().item()
            total_train += labels.size(0) * labels.size(1)

            tepoch.set_postfix(loss=total_loss.item())

    epoch_duration = time.time() - epoch_start_time
    print(f"Epoch {epoch+1} completed in {epoch_duration:.2f} seconds.")
    print(f"Epoch Loss: {running_loss / len(train_loader)}")
    train_accuracy = correct_train / total_train
    print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
    train_losses.append(running_loss / len(train_loader))

    # Validation loop
    alexnet_model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} Validation", leave=False):
            images, labels = images.to(device), labels.to(device)
            ec, vc, mf, he = alexnet_model(images)

            pred1 = torch.sigmoid(ec)
            pred2 = torch.sigmoid(vc)
            pred3 = torch.sigmoid(mf)
            pred4 = torch.sigmoid(he)

            correct_val += (pred1 > 0.5).float().eq(labels[:, 0].unsqueeze(1)).sum().item()
            correct_val += (pred2 > 0.5).float().eq(labels[:, 1].unsqueeze(1)).sum().item()
            correct_val += (pred3 > 0.5).float().eq(labels[:, 2].unsqueeze(1)).sum().item()
            correct_val += (pred4 > 0.5).float().eq(labels[:, 3].unsqueeze(1)).sum().item()
            total_val += labels.size(0) * 4

    val_accuracy = correct_val / total_val
    print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
    val_accuracies.append(val_accuracy)

# Final stats
total_training_time = time.time() - start_time
print(f"\nTotal Training Time: {total_training_time:.2f} seconds ({total_training_time / 60:.2f} minutes)")

# Plotting loss and accuracy
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', color='blue')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='green')
plt.title('Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Define sentiment categories
sentiment_categories = ["Emotion Through Color", "Visual Complexity", "Movement and Flow", "Facial Expressions and Human Emotion"]

# Set the model to evaluation mode
alexnet_model.eval()

# Initialize variables to hold predictions and ground truths
all_predictions = []
all_labels = []

# Iterate over the test dataset
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass through the model
        emotion_color, visual_complexity, movement_flow, human_emotion = alexnet_model(images)

        # Convert logits to probabilities using sigmoid
        pred_emotion_color = torch.sigmoid(emotion_color)
        pred_visual_complexity = torch.sigmoid(visual_complexity)
        pred_movement_flow = torch.sigmoid(movement_flow)
        pred_human_emotion = torch.sigmoid(human_emotion)

        # Binarize predictions (thresholding at 0.5)
        bin_emotion_color = (pred_emotion_color > 0.5).float()
        bin_visual_complexity = (pred_visual_complexity > 0.5).float()
        bin_movement_flow = (pred_movement_flow > 0.5).float()
        bin_human_emotion = (pred_human_emotion > 0.5).float()

        # Concatenate predictions for multi-label output
        batch_preds = torch.cat([bin_emotion_color, bin_visual_complexity, bin_movement_flow, bin_human_emotion], dim=1)
        all_predictions.append(batch_preds)
        all_labels.append(labels)

# Convert to NumPy arrays
all_predictions = torch.cat(all_predictions, dim=0).cpu().numpy()
all_labels = torch.cat(all_labels, dim=0).cpu().numpy()

# Calculate accuracy
accuracy = (all_predictions == all_labels).sum() / all_labels.size

# Compute evaluation metrics
precision = precision_score(all_labels, all_predictions, average='samples')
recall = recall_score(all_labels, all_predictions, average='samples')
f1 = f1_score(all_labels, all_predictions, average='samples')

# Multi-label confusion matrices
mcm = multilabel_confusion_matrix(all_labels, all_predictions)

# Plot confusion matrices for each class
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
for i, ax in enumerate(axes.flatten()):
    sns.heatmap(mcm[i], annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred Negative', 'Pred Positive'],
                yticklabels=['True Negative', 'True Positive'],
                ax=ax)
    ax.set_title(f'Confusion Matrix for {sentiment_categories[i]}')

plt.tight_layout()
plt.show()

# Display evaluation metrics
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
